# Install Libraries

In [40]:
! pip install -q transformers accelerate bitsandbytes peft torch langchain sentence-transformers faiss-cpu langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━

# Import Libraries

In [50]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

# Global Variables

In [7]:
INPUT_BASE_PATH = "/kaggle/input/"

In [26]:
NEWS_DATASET_PATH = os.path.join(INPUT_BASE_PATH, "european-commission-news/eu_commission_news.csv")

In [10]:
LLAMA_MODEL_PATH = os.path.join(INPUT_BASE_PATH, "llama-3.1/transformers/8b-instruct/2")

In [12]:
OUTPUT_BASE_PATH = '/kaggle/working/'

In [33]:
CLEAN_DATASET_PATH = os.path.join(OUTPUT_BASE_PATH, "eu_commission_news_clean.csv")

In [37]:
CHUNKS_DATASET_PATH = os.path.join(OUTPUT_BASE_PATH, "eu_commission_news_chunks.csv")

# Import Dataset

In [29]:
df = pd.read_csv(NEWS_DATASET_PATH)

In [30]:
df.head()

,title,link,date,summary,description
0,Mounting risks threaten survival of wild Europ...,https://environment.ec.europa.eu/news/mounting...,11 October 2025,Nearly 100 additional wild bee species in Euro...,"Abu Dhabi, United Arab Emirates, 11 October 20..."
1,Energy Efficiency Directive: Advancing the EU’...,https://energy.ec.europa.eu/news/energy-effici...,10 October 2025,Tomorrow (11 October) marks the deadline for E...,Tomorrow (11 October) marks the deadline for E...
2,EU’s Leading Role Instrumental in advancing su...,https://research-and-innovation.ec.europa.eu/n...,10 October 2025,G20 reinforces international collaboration in ...,"The G20 Research, Science and Innovation Minis..."
3,President von der Leyen travels to the Western...,https://enlargement.ec.europa.eu/news/presiden...,10 October 2025,"The President of the European Commission, Ursu...","The President of the European Commission, Ursu..."
4,Fishing vessel engine power - new Commission g...,https://oceans-and-fisheries.ec.europa.eu/news...,10 October 2025,The European Commission has published two tech...,The European Commission has published two tech...


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        266 non-null    object
 1   link         266 non-null    object
 2   date         266 non-null    object
 3   summary      266 non-null    object
 4   description  266 non-null    object
dtypes: object(5)
memory usage: 10.5+ KB


# Data Preparation & Cleaning

In [34]:
# Define a function to clean text
def clean_text(text):
    if pd.isna(text):
        return ""
    # Remove HTML tags
    text = BeautifulSoup(str(text), "html.parser").get_text()
    # Remove non-UTF characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Remove extra spaces and newlines
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning to relevant text columns
text_columns = ['title', 'summary', 'description']  # adjust based on your actual columns
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].apply(clean_text)

# Save the cleaned data
df.to_csv(CLEAN_DATASET_PATH, index=False)

# Preview cleaned data
df.head()

,title,link,date,summary,description
0,Mounting risks threaten survival of wild Europ...,https://environment.ec.europa.eu/news/mounting...,11 October 2025,Nearly 100 additional wild bee species in Euro...,"Abu Dhabi, United Arab Emirates, 11 October 20..."
1,Energy Efficiency Directive: Advancing the EU ...,https://energy.ec.europa.eu/news/energy-effici...,10 October 2025,Tomorrow (11 October) marks the deadline for E...,Tomorrow (11 October) marks the deadline for E...
2,EU s Leading Role Instrumental in advancing su...,https://research-and-innovation.ec.europa.eu/n...,10 October 2025,G20 reinforces international collaboration in ...,"The G20 Research, Science and Innovation Minis..."
3,President von der Leyen travels to the Western...,https://enlargement.ec.europa.eu/news/presiden...,10 October 2025,"The President of the European Commission, Ursu...","The President of the European Commission, Ursu..."
4,Fishing vessel engine power - new Commission g...,https://oceans-and-fisheries.ec.europa.eu/news...,10 October 2025,The European Commission has published two tech...,The European Commission has published two tech...


# Text Chunking - Using LangChain

In [38]:
# Combine relevant text fields into one
df["text"] = df[["title", "summary", "description"]].fillna("").agg(" ".join, axis=1)

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,      # ~300-400 tokens
    chunk_overlap=100,    # preserve context
    length_function=len
)

# Create chunks
chunks = []
for i, row in df.iterrows():
    splits = text_splitter.split_text(row["text"])
    for j, chunk in enumerate(splits):
        chunks.append({
            "id": f"{i}_{j}",
            "source_id": i,
            "title": row.get("title", ""),
            "date": row.get("date", ""),
            "url": row.get("url", ""),
            "text_chunk": chunk
        })

# Convert to DataFrame
chunks_df = pd.DataFrame(chunks)

# Save chunks for embedding
chunks_df.to_csv(CHUNKS_DATASET_PATH, index=False)

print(f"✅ Created {len(chunks_df)} text chunks.")
chunks_df.head()

✅ Created 684 text chunks.


,id,source_id,title,date,url,text_chunk
0,0_0,0,Mounting risks threaten survival of wild Europ...,11 October 2025,,Mounting risks threaten survival of wild Europ...
1,0_1,0,Mounting risks threaten survival of wild Europ...,11 October 2025,,"reveal serious challenges, with threats mounti..."
2,0_2,0,Mounting risks threaten survival of wild Europ...,11 October 2025,,are at risk of extinction. That compares to 77...
3,0_3,0,Mounting risks threaten survival of wild Europ...,11 October 2025,,pollinators are largely dependent on tradition...
4,0_4,0,Mounting risks threaten survival of wild Europ...,11 October 2025,,"such as carpenter bees, are benefiting from wa..."


# Local Embedding

In [42]:
# Initialize local embedding model
# Use new embedding model (better)
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-small-v2")

# Create a FAISS vector store from text chunks
vectorstore = FAISS.from_texts(
    texts=chunks_df["text_chunk"].tolist(),
    embedding=embeddings,
    metadatas=[
        {"title": t, "date": d, "url": u}
        for t, d, u in zip(chunks_df["title"], chunks_df["date"], chunks_df["url"])
    ]
)

# Save FAISS index locally
vectorstore.save_local("faiss_index_e5")


print("✅ Local embeddings generated and FAISS index saved successfully.")

/tmp/ipykernel_37/452625227.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-small-v2")
2025-10-28 15:08:42.473927: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761664122.939499      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761664123.062208      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

✅ Local embeddings generated and FAISS index saved successfully.


# Retriever + LLM Integration

In [43]:
# Load FAISS vectorstore
vectorstore = FAISS.load_local(
    "faiss_index_e5",
    embeddings,
    allow_dangerous_deserialization=True
)


# Build retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # top-5 relevant chunks

In [45]:
# Lightweight open-source model
local_pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2", device_map="auto", max_new_tokens=256)

llm = HuggingFacePipeline(pipeline=local_pipe)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

query = "What are the key initiatives of the European Commission in digital policy?"
result = qa_chain.run(query)

print(result)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipykernel_37/2179208560.py:4: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=local_pipe)
/tmp/ipykernel_37/2179208560.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(query)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

EU at the forefront of AI-driven research and scientific innovation. At its centre is RAISE - the Resource for AI Science in Europe - a virtual European institute that pools and coordinates AI resources for developing AI and applying it in science. Strategic actions include Back in April 2025, the Commission launched the AI Continent Action Plan, a plan that set the path for Europe to become a global leader in AI. The Apply AI and the AI in Science strategies are the next step in delivering this ambition and in positioning the EU to accelerate the use of AI in key sectors and science. For more information Press release - Commission launches two strategies to speed up AI uptake in European industry and science Apply AI Strategy AI in Science Strategy

Commission seeks views on future of European Standardisation The Commission

In [47]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)

response = conversation_chain({"question": "What did the EU do about AI regulations?"})
print(response["answer"])

/tmp/ipykernel_37/686707197.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/tmp/ipykernel_37/686707197.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation_chain({"question": "What did the EU do about AI regulations?"})
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Keeping European industry and science at the forefront of AI Learn how two EU strategies are putting industry and science at the forefront of artificial intelligence helping Europe become the global leader in trustworthy AI Artificial Intelligence (AI) is transforming how businesses operate, reshaping public services, and revolutionising science. AI has the potential to improve our lives in many ways. As the global race to harness its potential heats up, the European Commission has put forward two strategies that will help Europe stay ahead in AI industry and science respectively. The Apply AI Strategy sets out how to speed up the use of AI in key industries and the public sector. It will be used to help the EU unlock its societal benefits - from more accurate healthcare diagnoses to enhancing the efficiency and accessibilit

# Qunatitative Evaluation (Retrieval Quality)

In [49]:
query = "What did the European Commission announce about AI policy?"
query_embedding = embeddings.embed_query(query)

# Retrieve top 5 chunks
docs = retriever.get_relevant_documents(query)

# Compute cosine similarity for each retrieved doc
for i, doc in enumerate(docs):
    doc_emb = embeddings.embed_query(doc.page_content)
    sim = cosine_similarity([query_embedding], [doc_emb])[0][0]
    print(f"Doc {i+1} similarity: {sim:.3f}")
    print(f"→ {doc.metadata.get('title')}")
    print()

Doc 1 similarity: 0.880
→ Keeping European industry and science at the forefront of AI

Doc 2 similarity: 0.865
→ Keeping European industry and science at the forefront of AI

Doc 3 similarity: 0.863
→ Boosting the use of AI will make strategic industries more competitive

Doc 4 similarity: 0.855
→ AI is a strategic tool to improve scientific research

Doc 5 similarity: 0.849
→ AI is a strategic tool to improve scientific research



/tmp/ipykernel_37/1486757188.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


# Qualitative Evaluation

In [ ]:
#### Define the evaluation questions
evaluation_questions = [
    "What did the European Commission announce about climate policy?",
    "What are the recent initiatives on digital transformation?",
    "Did the European Commission discuss AI regulations?",
    "What were the main economic measures mentioned recently?",
    "What did the EU say about energy and sustainability?"
]

# Create an empty list to store results
results = []

for q in evaluation_questions:
    print(f"\n🧩 Query: {q}")
    start = time.time()

    # Run the RAG QA chain (adjust name if you're using `conversation_chain`)
    answer = qa_chain.run(q)

    end = time.time()

    print("\n💬 Answer:")
    print(answer)
    print(f"\n⏱️ Response time: {end - start:.2f} sec")

    # Manual evaluation (simple numeric inputs)
    print("\nRate the answer from 1–5 for the following criteria:")
    rel = int(input("   Relevance (1=bad, 5=excellent): "))
    faith = int(input("   Faithfulness (1=hallucinates, 5=factual): "))
    comp = int(input("   Completeness (1=incomplete, 5=full): "))
    conc = int(input("   Conciseness (1=too verbose, 5=clear): "))

    # Append results
    results.append({
        "query": q,
        "answer": answer,
        "response_time": round(end - start, 2),
        "relevance": rel,
        "faithfulness": faith,
        "completeness": comp,
        "conciseness": conc
    })

# Convert to DataFrame
eval_df = pd.DataFrame(results)

# Compute averages
avg_scores = eval_df[["relevance", "faithfulness", "completeness", "conciseness"]].mean()
print("\n✅ Average Scores:")
print(avg_scores)

# Save to CSV for report
eval_df.to_csv("rag_qualitative_evaluation.csv", index=False)
print("\n📁 Results saved to rag_qualitative_evaluation.csv")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧩 Query: What did the European Commission announce about climate policy?

💬 Answer:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

updated National Energy and Climate Plan Today, the European Commission decided to refer Poland to the Court of Justice of the European Union for failing to submit its final updated integrated National Energy and Climate Plan (NECP) in line with the Regulation (EU) 2018/1999 on the Governance of the Energy Union and Climate Action. Under Article 14(2) of the Governance Regulation, all Member States are obliged to submit their final updated NECPs by 30 June 2024, taking into account the Commission's recommendations and individual assessments. Poland submitted its draft plan and the Commission published the corresponding assessment in April 2024, including recommendations on where the country should raise its ambitions in line with EU targets 

   Relevance (1=bad, 5=excellent):  4
   Faithfulness (1=hallucinates, 5=factual):  3
   Completeness (1=incomplete, 5=full):  5
   Conciseness (1=too verbose, 5=clear):  2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧩 Query: What are the recent initiatives on digital transformation?

💬 Answer:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Republic of Congo. By reducing transport times from 45 days to just one week, the project will improve regional integration, lower emissions, and unlock opportunities for local industries and communities. Similar partnerships are being advanced worldwide, ranging from the digital transition in Colombia and regional electricity integration in Guatemala, to digital connectivity in the Middle East via the MEDUSA Submarine Optical Fibre Cable, water security via the the Aqaba-Amman Water Desalination, cultural heritage protection in Iraq, and clean energy projects in South Africa, Togo and Mauritania. They also include strategic port modernisation in Cabo Verde, as well as the development of new food and health value chains in Africa and the Caribbea

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧩 Query: What were the main economic measures mentioned recently?

💬 Answer:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

between companies that restrict competition. However, under Article 101(3) of the TFEU, such agreements can be declared compatible with the Single Market, provided they contribute to improving the production or distribution of goods or to promoting technical or economic progress, while allowing consumers a fair share of the resulting benefits and without eliminating competition. In November 2024, the Commission published a Staff Working Document setting out the results of the evaluation of the current TTBER and Guidelines. The evaluation confirmed that these instruments remain useful and relevant, but it also highlighted areas for possible improvement in terms of legal certainty and the need to reflect market developments. Following the evaluation,

   Relevance (1=bad, 5=excellent):  4
   Faithfulness (1=hallucinates, 5=factual):  4
   Completeness (1=incomplete, 5=full):  4
   Conciseness (1=too verbose, 5=clear):  4


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧩 Query: What did the EU say about energy and sustainability?


In [52]:
# Example evaluation queries
evaluation_questions = [
    "What did the European Commission announce about climate policy?",
    "What are the recent initiatives on digital transformation?",
    "Did the European Commission discuss AI regulations?",
    "What were the main economic measures mentioned recently?",
    "What did the EU say about energy and sustainability?"
]

results = []

for query in evaluation_questions:
    print(f"\n🧩 Evaluating query: {query}")
    start = time.time()

    # Step 1. Retrieve documents
    retrieved_docs = retriever.get_relevant_documents(query)
    retrieved_texts = [doc.page_content for doc in retrieved_docs]
    combined_context = " ".join(retrieved_texts)

    # Step 2. Generate answer using the local RAG pipeline
    answer = qa_chain.run(query)

    end = time.time()

    # Step 3. Compute embeddings
    q_emb = embeddings.embed_query(query)
    a_emb = embeddings.embed_query(answer)
    c_emb = embeddings.embed_query(combined_context)

    # Step 4. Compute similarity scores
    sim_q_a = cosine_similarity([q_emb], [a_emb])[0][0]          # Relevance
    sim_c_a = cosine_similarity([c_emb], [a_emb])[0][0]          # Faithfulness
    conciseness = len(answer) / max(len(combined_context), 1)    # Conciseness proxy
    completeness = len(retrieved_texts) / 5                      # How many chunks retrieved (0–1)

    results.append({
        "query": query,
        "answer": answer,
        "relevance_score": round(sim_q_a, 3),
        "faithfulness_score": round(sim_c_a, 3),
        "conciseness_score": round(conciseness, 3),
        "completeness_score": round(completeness, 3),
        "response_time_sec": round(end - start, 2)
    })

# Convert to DataFrame
eval_auto_df = pd.DataFrame(results)

# Compute average scores
avg_scores = eval_auto_df[["relevance_score", "faithfulness_score", "conciseness_score", "completeness_score"]].mean()
print("\n✅ Average Automatic Evaluation Scores:")
print(avg_scores)

# Save results for report
eval_auto_df.to_csv("rag_auto_evaluation.csv", index=False)
print("\n📁 Saved: rag_auto_evaluation.csv")

# Display preview
eval_auto_df.head()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧩 Evaluating query: What did the European Commission announce about climate policy?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧩 Evaluating query: What are the recent initiatives on digital transformation?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧩 Evaluating query: What were the main economic measures mentioned recently?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧩 Evaluating query: What did the EU say about energy and sustainability?

✅ Average Automatic Evaluation Scores:
relevance_score       0.8406
faithfulness_score    0.9872
conciseness_score     1.1938
completeness_score    1.0000
dtype: float64

📁 Saved: rag_auto_evaluation.csv


,query,answer,relevance_score,faithfulness_score,conciseness_score,completeness_score,response_time_sec
0,What did the European Commission announce abou...,Use the following pieces of context to answer ...,0.844,0.990,1.150,1.0,21.91
1,What are the recent initiatives on digital tra...,Use the following pieces of context to answer ...,0.819,0.982,1.243,1.0,34.02
2,Did the European Commission discuss AI regulat...,Use the following pieces of context to answer ...,0.848,0.990,1.135,1.0,20.62
3,What were the main economic measures mentioned...,Use the following pieces of context to answer ...,0.831,0.987,1.248,1.0,34.60
4,What did the EU say about energy and sustainab...,Use the following pieces of context to answer ...,0.861,0.987,1.193,1.0,26.32


# Try Llama 3.1

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1. Define the correct local path for Version 2 (Note the '2' at the end)

# 2. Define the 4-bit quantization config (Crucial for VRAM on a T4 GPU)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normalized Float 4-bit (recommended)
    bnb_4bit_compute_dtype=torch.bfloat16, # Use bfloat16 for computation if supported (faster)
)

In [3]:
# 3. Load Tokenizer and Model
# Note: You may need to add trust_remote_code=True for some models, but it is often unnecessary 
# for officially hosted models.
tokenizer = AutoTokenizer.from_pretrained(LLAMA_MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    LLAMA_MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto"
)

# Set the padding token, which is often missing or incorrectly set for Llama models
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Llama 3.1 8B Model loaded successfully!")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Llama 3.1 8B Model loaded successfully!


In [4]:
# Define the conversation history. We'll use a simple system message and a user query.
messages = [
    # Optional: A system message to define the model's persona or rules.
    {"role": "system", "content": "You are a concise, factual European news assistant."},
    # The user's question to test the model.
    {"role": "user", "content": "Explain why the Euro zone was created in one short paragraph."}
]

In [5]:
# Apply the template to create the final prompt string (input_ids)
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, # Tells the tokenizer to add the final 'assistant' header
    return_tensors="pt"
).to(model.device) # Move the prompt tokens to the GPU where the model is loaded

In [7]:
# The messages list is already defined from your previous cell
# messages = [...]

# 1. Apply the template to create the final prompt *string* (not a tensor yet)
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False, # Important: Returns a string
    add_generation_prompt=True
)

In [8]:
# 2. Tokenize the resulting string to get the required dictionary of tensors
# This is where the dictionary containing 'input_ids' and 'attention_mask' is created.
input_dict = tokenizer(prompt, return_tensors="pt").to(model.device)

In [9]:
import torch

with torch.no_grad():
    outputs = model.generate(
        **input_dict, # Pass the dictionary of tensors here
        max_new_tokens=256,    
        do_sample=True,        
        temperature=0.7        
    )

# Decode and print the output
response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract only the assistant response
assistant_response = response_text.split("assistant\n")[-1].strip()

print("\n--- ASSISTANT RESPONSE ONLY ---")
print(assistant_response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- ASSISTANT RESPONSE ONLY ---
The Eurozone was created to facilitate economic integration among European countries. It began with the signing of the Maastricht Treaty in 1992, which established the European Monetary Union (EMU). The treaty aimed to create a single currency, the Euro, to promote economic unity, increase trade, and reduce transaction costs among participating countries. The Euro was introduced in 1999 and replaced the national currencies of participating countries in 2002.
